<a href="https://colab.research.google.com/github/tatooland/keras/blob/master/KERAS_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##define GOOGLE DRIVER downloader

In [0]:
!pip install -U -q PyDrive
import os
from zipfile import ZipFile
from shutil import copy
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

def downloadFromGDriver(fileId):
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)
  fileName = fileId + '.zip'
  downloaded = drive.CreateFile({'id': fileId})
  downloaded.GetContentFile(fileName)
  ds = ZipFile(fileName)
  ds.extractall()
  os.remove(fileName)
  print('Extracted zip file ' + fileName)


## define uncompress zip file processing

In [0]:
import os
from zipfile import ZipFile
def uncompressZipFile(fileName):
   ds = ZipFile(fileName)
   ds.extractall()
   os.remove(fileName)
   print("process complete")

##define image iterator

In [0]:
import os
def iteratorImgFolder(folder, callback):
  g = os.walk(folder)
  ary = []
  for path, _, fileList in g:
    for fileName in fileList:
      callback(os.path.join(path, fileName), ary)
  return ary

## define image reader


###download image data from google driver example:

fileId = "1Uy-VRPQlrjfPHkJlZEgxUH-_4vMI1CUn"

downloadFromGDriver(fileId)

uncompressZipFile("seg_pred.zip")

uncompressZipFile("seg_test.zip")

uncompressZipFile("seg_train.zip")

In [0]:
from keras.preprocessing.image import load_img, img_to_array
def readImgAsArray(fileName, ary):
  img = load_img(fileName)
  format_img = img_to_array(img)
  print(format_img.shape)
  ary.extend(format_img)

##define image pre processing

In [0]:
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

def readImgDataAsFlow(train_dir, test_dir, validation_dir, target_size=(150, 150)):
  trainGenerator = ImageDataGenerator(data_format="channels_last")
  testGenerator = ImageDataGenerator(data_format="channels_last")
  validationGenerator = ImageDataGenerator(data_format="channels_last")
  trainFlow = trainGenerator.flow_from_directory(train_dir, class_mode='categorical', target_size=target_size)
  testFlow = testGenerator.flow_from_directory(test_dir, class_mode='categorical', target_size=target_size)
  validationFlow = validationGenerator.flow_from_directory(validation_dir, class_mode='categorical', target_size=target_size)
  return trainFlow, testFlow, validationFlow
  

##define model

###DCGAN

Notes on Implementation
We trained a custom version of the SAGAN model using spectral normalization and self-attention. We used Tensorflow’s tf.keras and Eager execution.

The Generator takes a random vector z and generates 128x128 RGB images. All layers, including dense layers, use spectral normalization. Additionally, the generator uses batch normalization and ReLU activations. Also, it uses self-attention in between middle-to-high feature maps. Like in the original implementation, we placed the attention layer to act on feature maps with dimensions 32x32.

The discriminator also uses spectral normalization (all layers). It takes RGB image samples of size 128x128 and outputs an unscaled probability. It uses leaky ReLUs with an alpha parameter of 0.02. Like the generator, it also has a self-attention layer operating of feature maps of dimensions 32x32.

![](https://sthalles.github.io/assets/advanced_gans/model_architecture.png)


SAGAM model architecture
The goal is to minimize the hinge version of the adversarial loss. To do that, we trained the generator and discriminator in an alternating style using the Adam Optimizer.


![](https://sthalles.github.io/assets/advanced_gans/hinge_gan_loss.png)

###tensorflow code below:
```
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(7*7*256, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((7, 7, 256)))
    assert model.output_shape == (None, 7, 7, 256) # Note: None is the batch size

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    assert model.output_shape == (None, 7, 7, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 14, 14, 64)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    assert model.output_shape == (None, 28, 28, 1)

    return model
```

![](https://gluon.mxnet.io/_images/dcgan.png)

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten，Conv2DTranspose, BatchNormalization, LeakyReLU, Reshape
from keras.optimizers import SGD


#DCGAN
def defineDCGANModel():
  model = Sequential()
  #define fully connection
  model.add(Dense(4*4*1024, input_dim=100))
  model.add(BatchNormalization())
  model.add(LeakyReLU())
  
  #reshape linear output to cube shape
  model.add(Reshape((4, 4, 1024)))
  
  #define transpose convolution
  model.add(Conv2DTranspose(filters=512, ))
  
  
  
  
  
  
  